In [55]:
encodings = {
    'Residence_type': {'Rural': 0, 'Urban': 1},
    'smoking_status': {
        'Unknown': 0, 'formerly smoked': 1, 'never smoked': 2, 'smokes': 3
    },
    'Symptom': {
        'Abdominal pain': 0, 'Abdominal pain, Chest pain': 1, 'Abdominal pain, Difficulty breathing': 2,
        'Abdominal pain, Fever': 3, 'Abdominal pain, Headache': 4, 'Abdominal pain, Weakness': 5,
        'Chest pain': 6, 'Chest pain, Abdominal pain': 7, 'Chest pain, Difficulty breathing': 8,
        'Chest pain, Fever': 9, 'Chest pain, Headache': 10, 'Chest pain, Weakness': 11,
        'Difficulty breathing': 12, 'Difficulty breathing, Abdominal pain': 13, 'Difficulty breathing, Chest pain': 14,
        'Difficulty breathing, Fever': 15, 'Difficulty breathing, Headache': 16, 'Difficulty breathing, Weakness': 17,
        'Fever': 18, 'Fever, Abdominal pain': 19, 'Fever, Chest pain': 20, 'Fever, Difficulty breathing': 21,
        'Fever, Headache': 22, 'Fever, Weakness': 23, 'Headache': 24, 'Headache, Abdominal pain': 25,
        'Headache, Chest pain': 26, 'Headache, Difficulty breathing': 27, 'Headache, Fever': 28,
        'Headache, Weakness': 29, 'Weakness': 30, 'Weakness, Abdominal pain': 31,
        'Weakness, Chest pain': 32, 'Weakness, Difficulty breathing': 33,
        'Weakness, Fever': 34, 'Weakness, Headache': 35
    },
    'Consciousness': {'Awake': 0, 'Responds to Pain': 1, 'Unconscious': 2},
    'Risk Factors': {
        'Cancer': 0, 'Cancer, Cardiovascular disease': 1, 'Cancer, Diabetes': 2,
        'Cancer, Hypertension': 3, 'Cancer, None': 4, 'Cancer, kidney failure': 5,
        'Cardiovascular disease': 6, 'Cardiovascular disease, Cancer': 7, 'Cardiovascular disease, Diabetes': 8,
        'Cardiovascular disease, Hypertension': 9, 'Cardiovascular disease, None': 10,
        'Cardiovascular disease, kidney failure': 11, 'Diabetes': 12, 'Diabetes, Cancer': 13,
        'Diabetes, Cardiovascular disease': 14, 'Diabetes, Hypertension': 15, 'Diabetes, None': 16,
        'Diabetes, kidney failure': 17, 'Hypertension': 18, 'Hypertension, Cancer': 19,
        'Hypertension, Cardiovascular disease': 20, 'Hypertension, Diabetes': 21, 'Hypertension, None': 22,
        'Hypertension, kidney failure': 23, 'kidney failure': 24, 'kidney failure, Cancer': 25,
        'kidney failure, Cardiovascular disease': 26, 'kidney failure, Diabetes': 27,
        'kidney failure, Hypertension': 28, 'kidney failure, None': 29, 'none risk factor': 30
    },
    'Massive Bleeding': {False: 0, True: 1},
    'Respiratory Distress': {False: 0, True: 1}
}

In [56]:
# --- Feedback Loop System for Continuous Learning ---
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.ensemble import VotingClassifier, GradientBoostingRegressor
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, mean_squared_error
import os

In [57]:
# --- 1. Load Existing Data and Models ---
print("Loading existing data and models...")

# Load existing processed data
try:
    df_existing = pd.read_excel("/home/abdeldjalil-hani/Desktop/emergency-sorting-system/data-sets/4. Scored Classified Preprocessed DATA.xlsx")
    print(f"Loaded existing dataset with {len(df_existing)} records")
except FileNotFoundError:
    print("No existing dataset found, starting fresh")
    df_existing = pd.DataFrame()

# Load models
classifier = joblib.load("/home/abdeldjalil-hani/Desktop/emergency-sorting-system/saved-models/classifier_voting_model.pkl")
regressors = {
    'Resuscitation (L1)': joblib.load("/home/abdeldjalil-hani/Desktop/emergency-sorting-system/saved-models/regressor_level_Resuscitation (L1).pkl"),
    'Emergent (L2)': joblib.load("/home/abdeldjalil-hani/Desktop/emergency-sorting-system/saved-models/regressor_level_Emergent (L2).pkl"),
    'Urgent (L3)': joblib.load("/home/abdeldjalil-hani/Desktop/emergency-sorting-system/saved-models/regressor_level_Urgent (L3).pkl"),
    'Less Urgent (L4)': joblib.load("/home/abdeldjalil-hani/Desktop/emergency-sorting-system/saved-models/regressor_level_Less Urgent (L4).pkl"),
    'Non-Urgent (L5)': joblib.load("/home/abdeldjalil-hani/Desktop/emergency-sorting-system/saved-models/regressor_level_Non-Urgent (L5).pkl")
}

Loading existing data and models...
Loaded existing dataset with 6962 records


In [58]:
# --- 2. Define Data Processing Functions ---
def preprocess_new_data(new_patients):
    """Preprocess new patient data to match training format"""
    processed = []
    
    for patient in new_patients:
        # Make a copy to avoid modifying original
        p = patient.copy()
        
        # Encode categorical fields
        for col, mapping in encodings.items():
            if col in p:
                p[col] = mapping[p[col]]

        # Split blood pressure
        if 'Blood Pressure (mmHg)' in p:
            bp = p.pop('Blood Pressure (mmHg)').split('/')
            p['blood_pressure'] = int(bp[0])
            p['heart_pressure'] = int(bp[1])

        processed.append(p)
    
    # Convert to DataFrame with correct column order
    feature_order = [
        'age', 'gender', 'chest pain type', 'cholesterol', 'exercise angina',
        'plasma glucose', 'skin_thickness', 'bmi', 'hypertension', 'heart_disease',
        'Residence_type', 'smoking_status', 'Symptom', 'Temperature (¬∞C)',
        'Heart Rate (bpm)', 'Respiratory Rate (breaths/min)', 'SpO2 (%)', 'Glasgow Score',
        'Consciousness', 'Massive Bleeding', 'Respiratory Distress',
        'Risk Factors', 'blood_pressure', 'heart_pressure'
    ]
    
    return pd.DataFrame(processed)[feature_order]

In [59]:
print("\nAdding new patient data directly...")
new_patients = [
    # Patient 1 (ID 5249)
    {
        'age': 50,
        'gender': 1,
        'chest pain type': 3,
        'cholesterol': 210,
        'exercise angina': 1,
        'plasma glucose': 119,
        'skin_thickness': 60,
        'bmi': 18.7,
        'hypertension': 1,
        'heart_disease': 1,
        'Residence_type': 'Urban',
        'smoking_status': 'formerly smoked',
        'Symptom': 'Chest pain',
        'Temperature (¬∞C)': 37.1,
        'Heart Rate (bpm)': 144,
        'Respiratory Rate (breaths/min)': 38,
        'SpO2 (%)': 75,
        'Glasgow Score': 5,
        'Consciousness': 'Unconscious',
        'Massive Bleeding': True,
        'Respiratory Distress': True,
        'Risk Factors': 'Hypertension',
        'Blood Pressure (mmHg)': '115/70'
    },
    # Patient 2 (ID 5258)
    {
        'age': 80,
        'gender': 1,
        'chest pain type': 1,
        'cholesterol': 157,
        'exercise angina': 0,
        'plasma glucose': 102.5,
        'skin_thickness': 42,
        'bmi': 18.3,
        'hypertension': 1,
        'heart_disease': 1,
        'Residence_type': 'Urban',
        'smoking_status': 'Unknown',
        'Symptom': 'Chest pain',
        'Temperature (¬∞C)': 36.4,
        'Heart Rate (bpm)': 90,
        'Respiratory Rate (breaths/min)': 21,
        'SpO2 (%)': 95,
        'Glasgow Score': 12,
        'Consciousness': 'Awake',
        'Massive Bleeding': False,
        'Respiratory Distress': False,
        'Risk Factors': 'Hypertension',
        'Blood Pressure (mmHg)': '121/86'
    },
    # Patient 3 (ID 5260)
    {
        'age': 68,
        'gender': 0,
        'chest pain type': 2,
        'cholesterol': 179,
        'exercise angina': 0,
        'plasma glucose': 118.2,
        'skin_thickness': 56,
        'bmi': 28.5,
        'hypertension': 1,
        'heart_disease': 0,
        'Residence_type': 'Urban',
        'smoking_status': 'never smoked',
        'Symptom': 'Difficulty breathing',
        'Temperature (¬∞C)': 39.5,
        'Heart Rate (bpm)': 93,
        'Respiratory Rate (breaths/min)': 14,
        'SpO2 (%)': 97,
        'Glasgow Score': 14,
        'Consciousness': 'Awake',
        'Massive Bleeding': False,
        'Respiratory Distress': False,
        'Risk Factors': 'Diabetes, Hypertension',
        'Blood Pressure (mmHg)': '113/76'
    },
    # Patient 4 (ID 5267)
    {
        'age': 30,
        'gender': 1,
        'chest pain type': 1,
        'cholesterol': 172,
        'exercise angina': 0,
        'plasma glucose': 106.8,
        'skin_thickness': 23,
        'bmi': 19.7,
        'hypertension': 0,
        'heart_disease': 0,
        'Residence_type': 'Rural',
        'smoking_status': 'formerly smoked',
        'Symptom': 'Difficulty breathing',
        'Temperature (¬∞C)': 37.4,
        'Heart Rate (bpm)': 98,
        'Respiratory Rate (breaths/min)': 16,
        'SpO2 (%)': 98,
        'Glasgow Score': 14,
        'Consciousness': 'Awake',
        'Massive Bleeding': False,
        'Respiratory Distress': True,
        'Risk Factors': 'none risk factor',
        'Blood Pressure (mmHg)': '117/79'
    }
]

print(f"\nAdded {len(new_patients)} new patients to the processing queue")


Adding new patient data directly...

Added 4 new patients to the processing queue


In [60]:
# --- 4. Process and Classify New Patients ---
print("\nProcessing and classifying new patients...")
new_data = preprocess_new_data(new_patients)


Processing and classifying new patients...


In [61]:
# Predict emergency level
emergency_levels = classifier.predict(new_data)
new_data['Emergency_Level'] = emergency_levels

In [62]:
# Predict scores within each level
scores = []
for idx, row in new_data.iterrows():
    level = row['Emergency_Level']
    score = regressors[level].predict(row.drop('Emergency_Level').values.reshape(1, -1))[0]
    scores.append(score)
    
new_data['Score'] = scores

/home/abdeldjalil-hani/Desktop/emergency-sorting-system/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
  warnings.warn(
/home/abdeldjalil-hani/Desktop/emergency-sorting-system/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
  warnings.warn(
/home/abdeldjalil-hani/Desktop/emergency-sorting-system/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
  warnings.warn(
/home/abdeldjalil-hani/Desktop/emergency-sorting-system/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
  warning

In [63]:
new_data

,age,gender,chest pain type,cholesterol,exercise angina,plasma glucose,skin_thickness,bmi,hypertension,heart_disease,...,SpO2 (%),Glasgow Score,Consciousness,Massive Bleeding,Respiratory Distress,Risk Factors,blood_pressure,heart_pressure,Emergency_Level,Score
0,50,1,3,210,1,119.0,60,18.7,1,1,...,75,5,2,1,1,18,115,70,Non-Urgent (L5),105.252703
1,80,1,1,157,0,102.5,42,18.3,1,1,...,95,12,0,0,0,18,121,86,Less Urgent (L4),93.800818
2,68,0,2,179,0,118.2,56,28.5,1,0,...,97,14,0,0,0,15,113,76,Less Urgent (L4),95.232351
3,30,1,1,172,0,106.8,23,19.7,0,0,...,98,14,0,0,1,30,117,79,Less Urgent (L4),97.471703


In [64]:
# --- 5. Combine with Existing Data ---
print("\nCombining with existing dataset...")
if not df_existing.empty:
    # Ensure column order matches
    new_data = new_data[df_existing.columns]
    combined_data = pd.concat([df_existing, new_data], ignore_index=True)
else:
    combined_data = new_data

print(f"New dataset size: {len(combined_data)} records")


Combining with existing dataset...
New dataset size: 6966 records


In [65]:
# --- 6. Retrain Models ---
print("\nRetraining models with expanded dataset...")

# Prepare data
X = combined_data.drop(columns=['Emergency_Level', 'Score'])
y_cls = combined_data['Emergency_Level']
y_reg = combined_data['Score']

# Split data
X_train, X_test, y_cls_train, y_cls_test, y_reg_train, y_reg_test = train_test_split(
    X, y_cls, y_reg, test_size=0.2, random_state=42
)

# Retrain classification model
print("Retraining classifier...")
classifier = VotingClassifier(
    estimators=[
        ('svm', SVC(probability=True, random_state=42)),
        ('dt', DecisionTreeClassifier(random_state=42)),
        ('knn', KNeighborsClassifier())
    ],
    voting='soft'
)
classifier.fit(X_train, y_cls_train)

# Evaluate classifier
y_cls_pred = classifier.predict(X_test)
print("\nClassification Performance:")
print(classification_report(y_cls_test, y_cls_pred))

# Retrain regression models
print("\nRetraining regressors...")
new_regressors = {}

for level in sorted(combined_data['Emergency_Level'].unique()):
    print(f"Training regressor for {level}")
    X_class = X_train[y_cls_train == level]
    y_class = y_reg_train[y_cls_train == level]
    
    reg = GradientBoostingRegressor(random_state=42)
    reg.fit(X_class, y_class)
    new_regressors[level] = reg
    
    # Evaluate
    preds = reg.predict(X_class)
    print(f"RMSE for {level}: {np.sqrt(mean_squared_error(y_class, preds)):.2f}")


Retraining models with expanded dataset...
Retraining classifier...

Classification Performance:
                    precision    recall  f1-score   support

     Emergent (L2)       0.90      0.96      0.93       488
  Less Urgent (L4)       0.98      0.65      0.78       196
   Non-Urgent (L5)       0.96      0.96      0.96        74
Resuscitation (L1)       0.88      0.94      0.91       520
       Urgent (L3)       1.00      0.97      0.98       116

          accuracy                           0.91      1394
         macro avg       0.94      0.90      0.91      1394
      weighted avg       0.91      0.91      0.91      1394


Retraining regressors...
Training regressor for Emergent (L2)
RMSE for Emergent (L2): 3.34
Training regressor for Less Urgent (L4)
RMSE for Less Urgent (L4): 2.50
Training regressor for Non-Urgent (L5)
RMSE for Non-Urgent (L5): 1.38
Training regressor for Resuscitation (L1)
RMSE for Resuscitation (L1): 3.48
Training regressor for Urgent (L3)
RMSE for Urgen

In [66]:
# --- 7. Save Updated Data and Models ---
print("\nSaving updated data and models...")

# Save updated dataset
combined_data.to_excel("/home/abdeldjalil-hani/Desktop/emergency-sorting-system/data-sets/7. Updated DATA.xlsx", index=False)

# Save updated models
os.makedirs("/home/abdeldjalil-hani/Desktop/emergency-sorting-system/saved-models-updated", exist_ok=True)
joblib.dump(classifier, "/home/abdeldjalil-hani/Desktop/emergency-sorting-system/saved-models-updated/classifier_voting_model.pkl")

for level, model in new_regressors.items():
    joblib.dump(model, f"/home/abdeldjalil-hani/Desktop/emergency-sorting-system/saved-models-updated/regressor_level_{level}.pkl")

print("Feedback loop completed successfully!")


Saving updated data and models...
Feedback loop completed successfully!


In [67]:
combined_data['Emergency_Level'].value_counts()

Emergency_Level
Resuscitation (L1)    2673
Emergent (L2)         2468
Less Urgent (L4)       908
Urgent (L3)            573
Non-Urgent (L5)        344
Name: count, dtype: int64